In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3  955k    3 30954    0     0  24083      0  0:00:40  0:00:01  0:00:39 24107
 13  955k   13  130k    0     0  50734      0  0:00:19  0:00:02  0:00:17 50759
 19  955k   19  187k    0     0  55212      0  0:00:17  0:00:03  0:00:14 55239
 26  955k   26  249k    0     0  58456      0  0:00:16  0:00:04  0:00:12 58474
100  955k  100  955k    0     0   199k      0  0:00:04  0:00:04 --:--:--  210k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm.notebook import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0219, 0.0267, 0.0080,  ..., 0.0196, 0.0038, 0.0115],
        [0.0319, 0.0458, 0.0208,  ..., 0.0428, 0.0339, 0.0454],
        [0.0213, 0.0264, 0.0423,  ..., 0.0284, 0.0300, 0.0069],
        ...,
        [0.0342, 0.0123, 0.0275,  ..., 0.0495, 0.0395, 0.0134],
        [0.0248, 0.0111, 0.0252,  ..., 0.0459, 0.0048, 0.0387],
        [0.0080, 0.0472, 0.0481,  ..., 0.0316, 0.0205, 0.0059]])
item_factors.weight tensor([[0.0198, 0.0117, 0.0151,  ..., 0.0349, 0.0218, 0.0157],
        [0.0143, 0.0325, 0.0166,  ..., 0.0026, 0.0347, 0.0071],
        [0.0303, 0.0341, 0.0213,  ..., 0.0266, 0.0397, 0.0440],
        ...,
        [0.0179, 0.0394, 0.0180,  ..., 0.0392, 0.0187, 0.0233],
        [0.0304, 0.0323, 0.0069,  ..., 0.0014, 0.0323, 0.0429],
        [0.0393, 0.0137, 0.0109,  ..., 0.0024, 0.0193, 0.0383]])


In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / (len(losses)+1))

  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 0.0
iter #1 Loss: 0.0
iter #2 Loss: 0.0
iter #3 Loss: 0.0
iter #4 Loss: 0.0
iter #5 Loss: 0.0
iter #6 Loss: 0.0
iter #7 Loss: 0.0
iter #8 Loss: 0.0
iter #9 Loss: 0.0
iter #10 Loss: 0.0
iter #11 Loss: 0.0
iter #12 Loss: 0.0
iter #13 Loss: 0.0
iter #14 Loss: 0.0
iter #15 Loss: 0.0
iter #16 Loss: 0.0
iter #17 Loss: 0.0
iter #18 Loss: 0.0
iter #19 Loss: 0.0
iter #20 Loss: 0.0
iter #21 Loss: 0.0
iter #22 Loss: 0.0
iter #23 Loss: 0.0
iter #24 Loss: 0.0
iter #25 Loss: 0.0
iter #26 Loss: 0.0
iter #27 Loss: 0.0
iter #28 Loss: 0.0
iter #29 Loss: 0.0
iter #30 Loss: 0.0
iter #31 Loss: 0.0
iter #32 Loss: 0.0
iter #33 Loss: 0.0
iter #34 Loss: 0.0
iter #35 Loss: 0.0
iter #36 Loss: 0.0
iter #37 Loss: 0.0
iter #38 Loss: 0.0
iter #39 Loss: 0.0
iter #40 Loss: 0.0
iter #41 Loss: 0.0
iter #42 Loss: 0.0
iter #43 Loss: 0.0
iter #44 Loss: 0.0
iter #45 Loss: 0.0
iter #46 Loss: 0.0
iter #47 Loss: 0.0
iter #48 Loss: 0.0
iter #49 Loss: 0.0
iter #50 Loss: 0.0
iter #51 Loss: 0.0
iter #52 Loss: 0.0
ite

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0219, 0.0267, 0.0080,  ..., 0.0196, 0.0038, 0.0115],
        [0.0319, 0.0458, 0.0208,  ..., 0.0428, 0.0339, 0.0454],
        [0.0213, 0.0264, 0.0423,  ..., 0.0284, 0.0300, 0.0069],
        ...,
        [0.0342, 0.0123, 0.0275,  ..., 0.0495, 0.0395, 0.0134],
        [0.0248, 0.0111, 0.0252,  ..., 0.0459, 0.0048, 0.0387],
        [0.0080, 0.0472, 0.0481,  ..., 0.0316, 0.0205, 0.0059]])
item_factors.weight tensor([[0.0198, 0.0117, 0.0151,  ..., 0.0349, 0.0218, 0.0157],
        [0.0143, 0.0325, 0.0166,  ..., 0.0026, 0.0347, 0.0071],
        [0.0303, 0.0341, 0.0213,  ..., 0.0266, 0.0397, 0.0440],
        ...,
        [0.0179, 0.0394, 0.0180,  ..., 0.0392, 0.0187, 0.0233],
        [0.0304, 0.0323, 0.0069,  ..., 0.0014, 0.0323, 0.0429],
        [0.0393, 0.0137, 0.0109,  ..., 0.0024, 0.0193, 0.0383]])


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

c:\Users\Sahil Sonawane\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Toy Story (1995)
	 American Beauty (1999)
	 Dances with Wolves (1990)
	 Beauty and the Beast (1991)
	 Mrs. Doubtfire (1993)
	 Monty Python and the Holy Grail (1975)
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Babe (1995)
	 Twister (1996)
	 Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Cluster #1
	 Pulp Fiction (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Godfather, The (1972)
	 Saving Private Ryan (1998)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 Back to the Future (1985)
	 Ace Ventura: Pet Detective (1994)
	 Memento (2000)
Cluster #2
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Shrek (2001)
	 Die Hard: With a Vengeance (1995)
	 Indiana Jones and the Last Crusade (1989)
	 One Flew Over the Cuckoo's Nest (1975)
	 E.T. the Extra-Terrestrial (1982)
	 Net, The (1995)
	 Interview with the Vampire: The Vampire Chronicles (1994)
	 Outbreak (1995)
	 Avatar